## Install dependencies

In [ ]:
!pip install jupyter-dash sweetviz

In [ ]:
!pip install -U --pre pandas-profiling

## Mount google drive (optonal)

Google drive can be mounted and acceessed from Google Colab.


Then the Logger data file can be copied onto Google Colab server which executes faster

Alternative is to manually upload the Logger data file from local PC.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import shutil 
# Copy Logger datafile from Google drive. Amend path as required.
shutil.copy('/content/drive/MyDrive/Colab Notebooks/Data_Science_Project2/Master data science data1 Ver 2.xlsx', "/content/Master data science data1 Ver 2.xlsx")

'/content/Master data science data1 Ver 2.xlsx'

### Import libraries

In [4]:
import base64
import datetime
import io

from jupyter_dash import JupyterDash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State
import statsmodels.api as sm
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import tensorflow as tf
import math
import dash_table
import pandas as pd
import numpy as np
import sweetviz as sv
from pandas_profiling import ProfileReport
from pandas_profiling.utils.cache import cache_file

import dash
import flask
from pathlib import Path
import os


### Load Logger datafile into dashboard
Make sure you change this to match the filename of your Logger data file.

In [ ]:

# Change Filename below to new data set

#filename = "Master data science data1 Ver 2.xlsx"
filename = "Master data science data1_cutdown.xlsx"

def load_file(filename):
    dataset = pd.DataFrame()
    logger = pd.DataFrame()
    raw_xls = pd.ExcelFile(filename)
    SheetList = raw_xls.sheet_names
    for sheet in SheetList:
        if sheet != "Meta data":
            logger = raw_xls.parse(sheet)
            logger.dropna(inplace=True)
            if logger.columns[0] != "Logger":
                logger['Logger'] = sheet.lower()
            else:
                dataset = dataset.append(logger,sort=False)
    dataset["Date_Time"] = pd.to_datetime(dataset["Date"].dt.strftime('%Y-%m-%d').astype('str')) + pd.to_timedelta(dataset["Time"].astype('str').str[-8:])
    dataset.drop(["Date","Time"] , axis=1, inplace=True)
    #dataset.set_index(['Logger','Date_Time'], inplace=True)
    return dataset

dataset = load_file(filename)

## Dashboard Code

Once you run the code below, click on the link at the bottom of the page:-

"Dash app running on xxxxxxxxxxxxxxxxxxx"

In [6]:

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
app.config.suppress_callback_exceptions = True

app.layout = html.Div([
    dcc.Tabs(id="tabs", value='tab-2', children=[
        #dcc.Tab(label='Load Data', value='tab-1'),
        dcc.Tab(label='EDA', value='tab-2'),
        dcc.Tab(label='Boxplots', value='tab-3'),
        dcc.Tab(label='Seasonal Decompose', value='tab-4'),
        dcc.Tab(label='Fourier transform', value='tab-5'),
    ]),
    html.Div(id='tabs-content')
])


layout1 = html.Div([
    html.H3('App 1'),
    dcc.Dropdown(
        id='app-1-dropdown',
        options=[
            {'label': 'App 1 - {}'.format(i), 'value': i} for i in [
                'NYC', 'MTL', 'LA'
            ]
        ]
    ),
    html.Div(id='app-1-display-value')
])

fileUploadLayout = html.Div([
    dcc.Upload(
        id='upload-data',
        children=html.Div([
            'Drag and Drop or ',
            html.A('Select Files')
        ]),
        style={
            'width': '100%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px'
        },
        # Allow multiple files to be uploaded
        multiple=True
    ),
    dcc.Loading(
            id="loading-1",
            type="dot",
            fullscreen=False,
            children=html.Div(id='output-data-upload')
        ),
    html.Div(dcc.Checklist(
            options=[{'label': i, 'value': i} for i in dataset['Logger'].unique()#dataset.index.get_level_values(0).unique()
                    ])),
])



### Tab Layout callback ##################################################################

@app.callback(Output('tabs-content', 'children'),
              Input('tabs', 'value'))
def render_content(tab):
    if tab == 'tab-1':
        return fileUploadLayout
    elif tab == 'tab-2':
        return EDA1_Layout
    elif tab == 'tab-3':
        return BoxplotLayout
    elif tab == 'tab-4':
        return SeasonalDecomposeLayout
    elif tab == 'tab-5':
        return FourierLayout

###########################################################################################

### Exploritory Data Anal Layout ##################################################

#HERE = Path(__file__).parent
#my_report = None
EDA1_Layout = html.Div([
     html.Br(),
    html.Label('Click buttons to generate EDA'),
#     html.Button("report", id="button_EDA1", n_clicks=0),
#     html.Div(id='output-EDA2')
    
    html.Button('Generate EDA #1', id='submit-Sweetviz', n_clicks=0),
    html.Br(),
    dcc.Loading(
            id="loading-4",
            type="cube",
            fullscreen=True,
            children=html.Div(id='container-Sweetviz')
        ),
    html.Hr(),  # horizontal line
    html.Button('Generate EDA #2', id='submit-Pandas', n_clicks=0),
    html.Br(),
    dcc.Loading(
            id="loading-5",
            type="cube",
            fullscreen=True,
            children=html.Div(id='container-Pandas')
        ),
    #html.A(html.Label("Open EDA#1 HTML"), href="/get_report", target="_blank"),
    
    #html.A(html.Button("report"), href="/get_report", target="_blank"),
])

@app.server.route("/get_report1")
def get_report1():
    return flask.send_from_directory("./", "SWEETVIZ_REPORT.html")

@app.server.route("/get_report2")
def get_report2():
    return flask.send_from_directory("./", "water_quality_report.html")

    
@app.callback(
    Output('container-Sweetviz', 'children'),
    [Input('submit-Sweetviz', 'n_clicks')])
def update_sweetviz(n_clicks):
    if os.path.isfile('SWEETVIZ_REPORT.html'):
        msg = html.A(html.Label("Open EDA#1 HTML"), href="/get_report1", target="_blank")
    else:
        msg = ''
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]
    if 'submit-Sweetviz' in changed_id:
        my_report = sv.analyze(dataset)
        my_report.show_html(open_browser=False) # Default arguments will generate to "SWEETVIZ_REPORT.html"
        msg = html.A(html.Label("Open EDA#1 HTML"), href="/get_report1", target="_blank")
    return msg

@app.callback(
    Output('container-Pandas', 'children'),
    [Input('submit-Pandas', 'n_clicks')])
def update_pandas(n_clicks):
    if os.path.isfile('water_quality_report.html'):
        msg = html.A(html.Label("Open EDA#2 HTML"), href="/get_report2", target="_blank")
    else:
        msg = ''
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]
    if 'submit-Pandas' in changed_id:
        profile = ProfileReport(dataset, title='Pandas Profiling Report', explorative=True)
        profile.to_file(Path("water_quality_report.html"))
        msg = html.A(html.Label("Open EDA#2 HTML"), href="/get_report2", target="_blank")
    return msg

   


###################################################################################

### Seasonal Decompose Layout ##################################################

SeasonalDecomposeLayout = html.Div([
    html.Label('Select Logger(s) to Plot'),
    dcc.Dropdown(
        id='crossfilter-logger2',
        options=[{'label': i, 'value': i} for i in dataset['Logger'].unique()],
        value=dataset['Logger'].unique(),
        multi=True
    ),
    html.Label("Resample-Period:"),
    dcc.RadioItems(
        id='resample1', 
        options=[{'value': 'M', 'label': 'Month'},
                 {'value': 'W', 'label': 'Week'},
                 {'value': 'D', 'label': 'Day'},
                 {'value': 'H', 'label': 'Hour'},
                 ],
        value='M', 
        labelStyle={'display': 'inline-block'}
    ),
    html.Label("y-axis:"),
    dcc.RadioItems(
        id='y-axis2', 
        options=[{'value': x, 'label': x} 
                 for x in dataset.select_dtypes('number').columns],
        value=dataset.select_dtypes('number').columns[0], 
        labelStyle={'display': 'inline-block'}
    ),
    dcc.Loading(
            id="loading-2",
            type="cube",
            fullscreen=False,
            children=html.Div(dcc.Graph(id="seasonal-plot", style={'height': '80vh'}))
        ),
])

def make_seasonal(dataset, loggers, metric, resamplePer):
    c = px.colors.qualitative.Plotly
    c = dict(zip(dataset.index.get_level_values(0).unique(), c[:len(dataset.index.get_level_values(0).unique())]))
    fig = make_subplots(rows=4, cols=1, shared_xaxes=True)
    for logger in loggers:
        metricClean = dataset.loc[logger][metric].dropna()
        metricClean = metricClean.resample(resamplePer).mean()
        metricClean[metricClean.isna()] = metricClean.mean()
        if not metricClean.empty:
            decomposition = sm.tsa.seasonal_decompose(metricClean, model='additive',extrapolate_trend='freq')
            fig.add_trace(go.Scatter(x=decomposition.observed.index, y=decomposition.observed, name=logger,
                                     legendgroup=logger, marker_color=c[logger]), row=1, col=1)
            fig.add_trace(go.Scatter(x=decomposition.trend.index, y=decomposition.trend, name=logger, 
                                     legendgroup=logger, showlegend=False, marker_color=c[logger]), row=2, col=1)
            fig.add_trace(go.Scatter(x=decomposition.seasonal.index, y=decomposition.seasonal, name=logger,
                                    legendgroup=logger, showlegend=False, marker_color=c[logger]), row=3, col=1)
            fig.add_trace(go.Scatter(x=decomposition.resid.index, y=decomposition.resid, name=logger,
                                     legendgroup=logger, showlegend=False, marker_color=c[logger]), row=4, col=1)
                
    fig.update_layout(
        title=f'Additive Decompose <br> {metric}',
        title_x=0.5,
        yaxis_title='Observed', yaxis2_title='Trend', yaxis3_title='Seasonal', yaxis4_title='Residual',
        xaxis4_title='Date',
        yaxis=dict(gridcolor='rgb(255, 255, 255)', linecolor='rgb(0, 0, 0)', mirror=True),
        yaxis2=dict(gridcolor='rgb(255, 255, 255)', linecolor='rgb(0, 0, 0)', mirror=True),
        yaxis3=dict(gridcolor='rgb(255, 255, 255)', linecolor='rgb(0, 0, 0)', mirror=True),
        yaxis4=dict(gridcolor='rgb(255, 255, 255)', linecolor='rgb(0, 0, 0)', mirror=True),
        xaxis=dict(gridcolor='rgb(255, 255, 255)', linecolor='rgb(0, 0, 0)', mirror=True),
        xaxis2=dict(gridcolor='rgb(255, 255, 255)', linecolor='rgb(0, 0, 0)', mirror=True),
        xaxis3=dict(gridcolor='rgb(255, 255, 255)', linecolor='rgb(0, 0, 0)', mirror=True),
        xaxis4=dict(gridcolor='rgb(255, 255, 255)', linecolor='rgb(0, 0, 0)', mirror=True),
        paper_bgcolor='rgb(255, 255, 255)',
        plot_bgcolor='rgb(255, 255, 255)',
        #showlegend=True
    )
    return fig

@app.callback(
    Output("seasonal-plot", "figure"), 
    Input("crossfilter-logger2", "value"),
    Input("y-axis2", "value"),
    Input("resample1", "value"))
def generate_SeasonalPlot(loggers, col, freq):
    if type(loggers) == str:
        loggers = [loggers]
    fig = make_seasonal(dataset=dataset.set_index(['Logger','Date_Time']), loggers=loggers, 
                        metric=col, resamplePer=freq)
    return fig

#################################################################################


### Fourier transform Layout ####################################################

FourierLayout = html.Div([
    html.Label('Select Logger(s) to Plot'),
    dcc.Dropdown(
        id='crossfilter-logger3',
        options=[{'label': i, 'value': i} for i in dataset['Logger'].unique()],
        value=dataset['Logger'].unique(),
        multi=True
    ),
    html.Label("y-axis:"),
    dcc.RadioItems(
        id='y-axis3', 
        options=[{'value': x, 'label': x} 
                 for x in dataset.select_dtypes('number').columns],
        value=dataset.select_dtypes('number').columns[0], 
        labelStyle={'display': 'inline-block'}
    ),
    dcc.Loading(
            id="loading-3",
            type="cube",
            fullscreen=False,
            children=html.Div(dcc.Graph(id="fourier-plot", style={'height': '80vh'}))
        ),
])


def make_fourier(dataset, loggers, metric):
    pltRows = math.ceil(len(dataset.index.get_level_values(0).unique())/2)
    pltCols = 2 if len(dataset.index.get_level_values(0).unique()[0]) > 1 else 1
    #c = px.colors.qualitative.Plotly
    #c = dict(zip(dataset.index.get_level_values(0).unique(), c[:len(dataset.index.get_level_values(0).unique())]))
    fig = make_subplots(rows=pltRows, cols=pltCols, shared_xaxes=False, 
                        subplot_titles=dataset.index.get_level_values(0).unique())
    row = 1
    col = 1
    for logger in loggers:
        metricClean = dataset.loc[logger][metric].dropna()
        metricClean = metricClean.resample('60min').mean()
        metricClean.dropna(inplace=True)
        if not metricClean.empty:
            fft = tf.signal.rfft(metricClean)
            f_per_dataset = np.arange(0, len(fft))
            n_samples_h = len(metricClean)
            hours_per_year = 24*365.2524
            years_per_dataset = n_samples_h/(hours_per_year)
            f_per_year = f_per_dataset/years_per_dataset
            mask = (f_per_year >= 0.5) & (f_per_year <= hours_per_year)
            fig.add_trace(go.Scatter(x=f_per_year[mask], y=np.abs(fft)[mask], name=logger,
                                     line_shape='hv', legendgroup=logger), row=row, col=col)
            if col == 2:
                row += 1
                col = 1
            else:
                 col = 2   
            
    fig.update_xaxes(type="log")
    fig.update_xaxes(
    ticktext=['1/Year', '1/Month', '1/Week', '1/day', '12hours', '6hours'],
    tickvals=[1, 12, 52, 365.2524, hours_per_year/12, hours_per_year/6],
    gridcolor='rgb(255, 255, 255)',
    linecolor='rgb(0, 0, 0)', mirror=True)
    fig.update_yaxes(
    gridcolor='rgb(255, 255, 255)',
    linecolor='rgb(0, 0, 0)', mirror=True)
    
    fig.update_layout(
        title=f'Fourier transform <br> {metric}',
        title_x=0.5,
        #xaxis_title='Frequency (log scale)',
        paper_bgcolor='rgb(255, 255, 255)',
        plot_bgcolor='rgb(255, 255, 255)',
        #showlegend=True
    )
    return fig

@app.callback(
    Output("fourier-plot", "figure"), 
    Input("crossfilter-logger3", "value"),
    Input("y-axis3", "value"))
def generate_FourierPlot(loggers, col):
    if type(loggers) == str:
        loggers = [loggers]
    fig = make_fourier(dataset=dataset.set_index(['Logger','Date_Time']), loggers=loggers, 
                        metric=col)
    return fig

##########################################################################################

    
def parse_contents(contents, filename, date):
    global dataset
    #dataset = pd.DataFrame()
    content_type, content_string = contents.split(',')

    decoded = base64.b64decode(content_string)
    try:
        if 'csv' in filename:
            # Assume that the user uploaded a CSV file
            df = pd.read_csv(
                io.StringIO(decoded.decode('utf-8')))
        elif 'xlsx' in filename:
            # Assume that the user uploaded an excel file
            #df = pd.read_excel(io.BytesIO(decoded))
            raw_xls = pd.ExcelFile(io.BytesIO(decoded))
            SheetList = raw_xls.sheet_names
            #dataset = pd.DataFrame()
            logger = pd.DataFrame()
            for sheet in SheetList:
                if sheet != "Meta data":
                    logger = raw_xls.parse(sheet)
                    logger.dropna(inplace=True)
                    if logger.columns[0] != "Logger":
                        logger['Logger'] = sheet.lower()
                    else:
                        dataset = dataset.append(logger,sort=False)
            dataset["Date_Time"] = pd.to_datetime(dataset["Date"].dt.strftime('%Y-%m-%d').astype('str')) + pd.to_timedelta(dataset["Time"].astype('str').str[-8:])
            dataset.drop(["Date","Time"] , axis=1, inplace=True)
            dataset.set_index(['Logger','Date_Time'], inplace=True)

    except Exception as e:
        print(e)
        return html.Div([
            'There was an error processing this file.'
        ])

    return html.Div([
        html.H5(filename),
        html.H6(datetime.datetime.fromtimestamp(date)),

        html.Hr(),  # horizontal line

    ])


@app.callback(Output('output-data-upload', 'children'),
              Input('upload-data', 'contents'),
              State('upload-data', 'filename'),
              State('upload-data', 'last_modified'))

def update_output(list_of_contents, list_of_names, list_of_dates):
    if list_of_contents is not None:
        children = [
            parse_contents(c, n, d) for c, n, d in
            zip(list_of_contents, list_of_names, list_of_dates)]
        return children


### Boxplot Layout #################################################################

BoxplotLayout = html.Div([
    html.P("Logger:"),
    dcc.Dropdown(
                id='crossfilter-logger',
                options=[{'label': i, 'value': i} for i in dataset['Logger'].unique()],
                value=dataset['Logger'].unique()[0]
            ),
    html.P("y-axis:"),
    dcc.RadioItems(
        id='y-axis', 
        options=[{'value': x, 'label': x} 
                 for x in dataset.select_dtypes('number').columns],
        value=dataset.select_dtypes('number').columns[0], 
        labelStyle={'display': 'inline-block'}
    ),
    dcc.Loading(
            id="loading-1",
            type="cube",
            fullscreen=False,
            children=html.Div(dcc.Graph(id="box-plot", style={'height': '80vh'}))
        ),
])

def make_boxplot(logger, y):
    Temp = dataset.loc[dataset["Logger"]==logger].copy()
    Temp['value']= Temp[y]

    # Prepare data
    Temp['year'] = [d.year for d in Temp.Date_Time]
    Temp['month'] = [d.strftime('%b') for d in Temp.Date_Time]
    Temp['day'] = [d.strftime('%a') for d in Temp.Date_Time]
    Temp['hour'] = [d.strftime('%H') for d in Temp.Date_Time]
    #years = Temp['year'].unique()

    fig = make_subplots(rows=2, cols=2, subplot_titles=['Year-wise Box Plot\n(The Trend)', 
                                                        'Month-wise Box Plot\n(The Seasonality)',
                                                       'Day-wise Box Plot',
                                                       'Hour-wise Box Plot'], 
                        shared_xaxes=False)

    for year_num in Temp['year'].unique():
        fig.add_trace(go.Box(x=Temp.loc[Temp['year']==year_num]['year'], y=Temp.loc[Temp['year']==year_num]['value'],
         name=str(year_num)), row=1, col=1)
    
    for month_num in Temp['month'].unique():
        fig.add_trace(go.Box(x=Temp.loc[Temp['month']==month_num]['month'], y=Temp.loc[Temp['month']==month_num]['value'],
         name=str(month_num)), row=1, col=2)
    
    for day_num in Temp['day'].unique():
        fig.add_trace(go.Box(x=Temp.loc[Temp['day']==day_num]['day'], y=Temp.loc[Temp['day']==day_num]['value'],
         name=str(day_num)), row=2, col=1)
        
    for hour_num in Temp['hour'].unique():
        fig.add_trace(go.Box(x=Temp.loc[Temp['hour']==hour_num]['hour'], y=Temp.loc[Temp['hour']==hour_num]['value'],
         name=str(hour_num)), row=2, col=2)
    
    fig.update_layout(
        #title='Year-wise Box Plot\n(The Trend)',
        #title_x=0.5,
        yaxis_title=y, yaxis2_title=y, yaxis3_title=y, yaxis4_title=y,
        xaxis_title='Year', xaxis2_title='Month', xaxis3_title='Day', xaxis4_title='Hour',
        yaxis=dict(gridcolor='rgb(255, 255, 255)', linecolor='rgb(0, 0, 0)', mirror=True),
        yaxis2=dict(gridcolor='rgb(255, 255, 255)', linecolor='rgb(0, 0, 0)', mirror=True),
        yaxis3=dict(gridcolor='rgb(255, 255, 255)', linecolor='rgb(0, 0, 0)', mirror=True),
        yaxis4=dict(gridcolor='rgb(255, 255, 255)', linecolor='rgb(0, 0, 0)', mirror=True),
        xaxis=dict(gridcolor='rgb(255, 255, 255)', linecolor='rgb(0, 0, 0)', mirror=True),
        xaxis2=dict(gridcolor='rgb(255, 255, 255)', linecolor='rgb(0, 0, 0)', mirror=True),
        xaxis3=dict(gridcolor='rgb(255, 255, 255)', linecolor='rgb(0, 0, 0)', mirror=True),
        xaxis4=dict(gridcolor='rgb(255, 255, 255)', linecolor='rgb(0, 0, 0)', mirror=True),
        paper_bgcolor='rgb(255, 255, 255)',
        plot_bgcolor='rgb(255, 255, 255)',
        showlegend=False,
        #height=1000
    )
    return fig

    
@app.callback(
    Output("box-plot", "figure"), 
    [Input("crossfilter-logger", "value"), 
     Input("y-axis", "value")])
def generate_Boxplot(logger, y):
    fig = make_boxplot(logger=logger, y=y)
    return fig

################################################################################################

# Run app and display result inline in the notebook
#app.run_server(mode='inline', debug=True)
app.run_server()

Dash app running on http://127.0.0.1:8050/
